# Import libraries and define essential functions

In [ ]:
%load_ext autoreload
%autoreload 2

import itertools
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import plotly.graph_objects as pgo
import plotly.subplots as psub
import plotly.express as px
from pathlib import Path
from plotly.subplots import make_subplots
from Utils.plotting_funcs import plotly_nx

np.set_printoptions(precision=3, linewidth=120, suppress=True)
layout_width = 800

In [ ]:
def get_laplacian(A):
    """Return the Laplacian matrix, and its eigenvalues and eigenvectors."""
    D = np.diag(A.sum(1))
    L = D - A
    lambdas, vectors = np.linalg.eigh(L)
    sort = lambdas.argsort()
    lambdas = lambdas[sort]
    vectors = vectors[:, sort]
    return lambdas, vectors, L


def get_fiedler(lambdas, vectors, *args):  # *args is here so we can ignore L from get_laplacian
    """Return the Fiedler vector and its corresponding eigenvalue with multiplicity."""
    l2 = lambdas[1]
    f = vectors[:, 1]
    l2_multiplicity = np.count_nonzero(np.isclose(lambdas, l2))
    return round(l2, 5), l2_multiplicity, f

In [ ]:
def plot_graphs(graphs, figsize=14, dotsize=20):
    """
    Utility to plot a lot of graphs from an array of graphs.
    Each graphs is a list of edges; each edge is a tuple.
    """
    fig = plt.figure(figsize=(figsize, figsize))
    fig.patch.set_facecolor("white")  # To make copying possible (no transparent background)
    k = int(np.sqrt(len(graphs)))
    for i, g in enumerate(graphs):
        plt.subplot(k + 1, k + 1, i + 1)
        G = nx.from_numpy_array(graphs[g])
        nx.draw_kamada_kawai(G, node_size=dotsize)
        l2, mul, _ = get_fiedler(*get_laplacian(graphs[g]))
        plt.title(f"Graph {g} - {len(G.edges)}\nl2={l2} (x{mul})")
        print(".", end="")


# plot_graphs([[(0,1),(1,2),(1,3)]])

In [ ]:
def calc_thresholds_combs(A, L, lambdas, vectors, func=None, addremove=1, p=False):
    """Calculate the current lambda_2 and K_lambda_2 and return them."""
    l2, l2_multiplicity, f = get_fiedler(lambdas, vectors)

    K_l2 = 0
    s_val = 0
    pairs = None
    if func is not None:
        search = np.empty_like(A)
        search[:] = np.nan
        for i, j in itertools.combinations(range(len(f)), 2):
            if abs(f[i] - f[j]) > 10e-5 and ((addremove == 1 and A[i][j] == 0) or (addremove == -1 and A[i][j] == 1)):
                search[i][j] = addremove * (f[i] - f[j]) ** 2

            if A[i][j] == 1:
                K_l2 = max(K_l2, (f[i] - f[j]) ** 2)

        s_val = func(search)
        if l2_multiplicity == 1 or addremove == -1:
            pairs = {tuple(p) for p in np.argwhere(np.isclose(search, s_val))}
        else:
            pairs = f"Unknown, at least {l2_multiplicity} links"

    if p:
        print(f"({l2:.5f}, {K_l2:.5f})")
        print(f"L  = {np.array2string(L, prefix='L  = ')}")
        print(f"eval={np.array2string(lambdas)}")
        print(f"evec={np.array2string(vectors, prefix='evec=')}")
        print("-----------------------------------------------\n")

    return round(K_l2 * 0.2, 5), pairs

# Load graphs and plot them

In [ ]:
from collections import OrderedDict
from my_graphs_dataset import GraphDataset

NV = 5  # Number of nodes in the graph.

selection = {NV: -1}
loader = GraphDataset(selection, suppress_output=True)

gs = OrderedDict({f"G{NV},{i}": nx.to_numpy_array(g) for i, g in enumerate(loader.graphs(raw=False))})

print(f"Drawing {len(gs)} graphs...")
plot_graphs(gs, figsize=30, dotsize=20)

### Use this cell to get MATLAB matrix of a selected graph

In [ ]:
import sys

G = gs['G5,10']
np.savetxt(sys.stdout, G, fmt='%d', delimiter=' ', newline=';\n')

# Possible values of lambda_2 depending on the number of edges
If we consider that all edges are unweighted, i.e., they are all equal to 1, lambda_2 will have discrete values between zero and the number of vertices in the graph. The number of these discrete values depends on the number of vertices. Here, we are interested in which values of lambda_2 are possible for a given number of edges and what is the minimum distance between any real value of lambda_2 and the nearest discrete value. We can then calculate an accaptable treshold K_lambda_2 within which the desired value of lambda_2 cannot be achieved with unweighted edges.

In [ ]:
from collections import defaultdict

# Prepara a dictionary to store the l2 values by number of edges.
l2_by_edges = defaultdict(list)

# Go through each graph, calculate the Fiedler value, and store it in the dictionary by number of edges.
for i, A in enumerate(gs.values()):
    l2 = get_fiedler(*get_laplacian(A))[0]
    l2_by_edges[np.count_nonzero(A == 1) / 2].append(l2)

In [ ]:
# Create a figure with 3 subplots (rows)
fig = make_subplots(rows=3, cols=1,
                    subplot_titles=[
                        "Possible λ₂ by number of edges",
                        "Needed K_λ₂ for specified λ₂",
                        "Possible values of λ₂",
                    ],
                    vertical_spacing=0.1)
axis_labels = sorted(l2_by_edges.keys())

# 1st subplot - bar chart of possible lambda_2 values by edge count
for edge_count in axis_labels:
    values = sorted(l2_by_edges[edge_count])
    num_values = len(values)

    # Plot each lambda_2 value for this edge count
    for i, value in enumerate(values):
        if value > 0:  # Only plot positive values
            fig.add_trace(
                pgo.Bar(
                    x=[edge_count],
                    y=[value],
                    text=f"{value:.3f}",
                    textposition="outside",
                    textangle=-90,
                    showlegend=False,
                    marker_color='blue',
                    width=0.8/num_values if num_values > 1 else 0.5,
                    offset=(-0.4 + 0.8*i/num_values) if num_values > 1 else 0
                ),
                row=1, col=1
            )

# 2nd subplot - line plot for minimum K_lambda_2
possible_l2 = sorted(v for k in l2_by_edges for v in l2_by_edges[k] if v > 0)
max_l2 = possible_l2[-1]
x_line = np.linspace(0, max_l2, 500)
y_line = []
i = 0
for xi in x_line:
    # If current xi is greater than the possible_l2, move to the next possible_l2
    while i < len(possible_l2)-1 and xi > possible_l2[i]:
        i += 1
    # Before the first possible_l2, possible_l2[i] must be greater than xi, so we just record the difference.
    if i == 0:
        y_line.append(possible_l2[i] - xi if i < len(possible_l2) else 0)
    # In other cases, we take the minimum of the difference between xi and the two possible_l2 values on left and right.
    else:
        y_line.append(min(xi - possible_l2[i-1], possible_l2[i] - xi) if i < len(possible_l2) else xi - possible_l2[i-1])

fig.add_trace(
    pgo.Scatter(
        x=x_line,
        y=y_line,
        mode='lines',
        line=dict(color='blue', width=2),
        name="Min K_lambda_2"
    ),
    row=2, col=1
)

# 3rd subplot - scatter plot of possible lambda_2 values
fig.add_trace(
    pgo.Scatter(
        x=possible_l2,
        y=[1] * len(possible_l2),
        mode='markers',
        marker=dict(size=10, color='blue'),
        name="Possible λ₂ values"
    ),
    row=3, col=1
)

# Update layout
fig.update_layout(
    height=900,
    showlegend=False,
)

# Update axes
fig.update_xaxes(title_text="Number of edges", row=1, col=1)
fig.update_yaxes(title_text="λ₂", row=1, col=1)

fig.update_xaxes(title_text="Reference λ₂", row=2, col=1)
fig.update_yaxes(title_text="Minimum K_λ₂", row=2, col=1)

fig.update_xaxes(title_text="λ₂", row=3, col=1)
fig.update_yaxes(showticklabels=False, row=3, col=1)

fig.show()

# Analyze lambda_2 sensitivity to changing link weights

In [ ]:
from collections import namedtuple

Res = namedtuple("Res", ["pair", "best"])


def first_approx_Ghosh(A, l2, f, addremove):
    """
    Implementation of heuristic from

    Ghosh, A., Boyd, S., 2006. Growing well-connected graphs, in: Proceedings
    of the IEEE Conference on Decision and Control. IEEE., pp. 6605-6611.
    https://doi.org/10.1109/CDC.2006.377282

    addremove: 1 to add an edge, -1 to remove an edge
    """
    pairs = None
    best = None

    if l2 > 0:  # We send l2=0 when l2 has multiplicity > 1
        search = np.zeros_like(A)
        search[:] = np.nan
        pairs = {}
        # Search through all pairs of nodes (non-existing edges for add, existing edges for remove).
        for i, j in itertools.combinations(range(len(f)), 2):
            if (addremove == 1 and A[i][j] == 0) or (addremove == -1 and A[i][j] == 1):
                approx = (f[i] - f[j]) ** 2
                pairs[(i, j)] = round(approx, 2) * addremove
                if abs(f[i] - f[j]) > 10e-5:
                    search[i][j] = approx * addremove

        # Find the pair of nodes that maximizes the approximation.
        s_val = np.nanmax(search)
        best = {tuple(p) for p in np.argwhere(np.isclose(search, s_val))}

    # if p:
    #     print(f"({l2:.5f}, {K_l2:.5f})")
    #     print(f"L  = {np.array2string(L, prefix='L  = ')}")
    #     print(f"eval={np.array2string(lambdas)}")
    #     print(f"evec={np.array2string(vectors, prefix='evec=')}")
    #     print('-----------------------------------------------\n')

    return Res(pairs, best)


def second_approx_He(A, addremove):
    """
    Implementation of second-order heuristic (13) from

    He, Z., 2019. Optimization of convergence rate via algebraic connectivity.

    addremove: 1 to add an edge, -1 to remove an edge
    """
    n = np.size(A, 1)

    D = np.diag(A.sum(1))
    Q = D - A

    eps = 0.5 / np.max(D)

    R = np.eye(n) - eps * Q - 1 / n * np.ones(n)

    lambdas, vectors = np.linalg.eigh(R)
    sort = lambdas.argsort()
    lambdas = lambdas[sort]
    vectors = vectors[:, sort]
    l1 = lambdas[-1]
    z = vectors[:, -1]

    search = np.empty_like(A)
    search[:] = np.nan
    pairs = {}
    # Search through all pairs of nodes (non-existing edges for add, existing edges for remove).
    for i, j in itertools.combinations(range(n), 2):
        # Create a perturbed adjacency matrix.
        if (addremove == 1 and A[i][j] == 0) or (addremove == -1 and A[i][j] == 1):
            dA = np.zeros_like(A)
            dA[i, j] = addremove
            dA[j, i] = addremove
        else:
            continue
        # Calculate the approximation and store it.
        dQ = np.diag(dA.sum(1)) - dA
        approx = z.T @ dQ @ z - (eps / l1) * z.T @ np.linalg.matrix_power(dQ, 2) @ z
        pairs[(i, j)] = round(approx, 2)
        if abs(approx) > 10e-5:
            search[i, j] = approx

    # Find the pair of nodes that maximizes the approximation.
    s_val = np.nanmax(search)
    best = {tuple(p) for p in np.argwhere(np.isclose(search, s_val))}

    return Res(pairs, best)


def exact_He(A, addremove):
    """
    Implementation of the exact formulation (8) from

    He, Z., 2019. Optimization of convergence rate via algebraic connectivity.

    addremove: 1 to add an edge, -1 to remove an edge
    """
    n = np.size(A, 1)

    Q = np.diag(A.sum(1)) - A

    lambdas, vectors = np.linalg.eigh(Q)
    sort = lambdas.argsort()
    lambdas = lambdas[sort]
    vectors = vectors[:, sort]
    lambdas = np.insert(lambdas, 0, -1)  # Insert an extra element so we can use 1-indexing
    vectors = np.insert(vectors, 0, -1, axis=1)

    sum_range = set(range(1, n + 1)) - {2}
    search = np.empty_like(A)
    search[:] = np.nan
    pairs = {}
    # Search through all pairs of nodes (non-existing edges for add, existing edges for remove).
    for i, j in itertools.combinations(range(n), 2):
        # Create a perturbed adjacency matrix.
        if (addremove == 1 and A[i][j] == 0) or (addremove == -1 and A[i][j] == 1):
            dA = np.zeros_like(A)
            dA[i, j] = addremove
            dA[j, i] = addremove
        else:
            continue
        # Calculate the approximation and store it.
        dQ = np.diag(dA.sum(1)) - dA
        approx = vectors[:, 2].T @ dQ @ vectors[:, 2] + sum(
            pow(vectors[:, k].T @ dQ @ vectors[:, 2], 2) / (lambdas[2] - lambdas[k]) for k in sum_range
        )
        pairs[(i, j)] = round(approx, 2)
        if abs(approx) > 10e-5:
            search[i, j] = approx

    # Find the pair of nodes that maximizes the approximation.
    s_val = np.nanmax(search)
    best = {tuple(p) for p in np.argwhere(np.isclose(search, s_val))}

    errors = None
    # for p in best:
    #     dA = np.zeros_like(A)
    #     dA[p[0], p[1]] = addremove
    #     dA[p[1], p[0]] = addremove
    #     dQ = np.diag(dA.sum(1)) - dA
    #     errors[p] = np.linalg.norm(dQ, 'fro') ** 3

    return Res(pairs, best), errors

The individual plots below show:
- graph on the left sides
- sensitivity of lambda_2 to changing link weights on the right side
- starting value of lambda_2 and corresponding Fiedler vector in the title
- below: the predicted value of change in lambda_2 when adding or removing a link using all three methods
  - the biggest (adding) or the smallest non-zero (removing) values are bolded

Solid lines represent links that are being added.  
Dashed lines represent links that are being removed.  
Markers (square, circle, x) represent the predicted value of new lambda_2 when adding or removing the best link using the three methods.

In [ ]:
from copy import copy


def analyze_sensitiviy_and_approximations(k, A_start, p=False):
    x = np.linspace(0, 1, 50)  # Discrete values to change the link weights.
    N = A_start.shape[0]       # Number of nodes in the graph.

    r_proto = {1: Res(None, None), -1: Res(None, None)}
    e_proto = {1: None, -1: None}
    symbols = {"1st": "square", "2nd": "circle", "exact": "x"}
    results = {"1st": copy(r_proto), "2nd": copy(r_proto), "exact": copy(r_proto)}
    errors = {"1st": copy(e_proto), "2nd": copy(e_proto), "exact": copy(e_proto)}

    # Initial lambda value and max change estimate according to Ghosh.
    l2_0, lambda_mul, f = get_fiedler(*get_laplacian(A_start))
    l2 = l2_0 if lambda_mul == 1 else 0
    results["1st"][1] = first_approx_Ghosh(A_start, l2, f, addremove=1)
    results["1st"][-1] = first_approx_Ghosh(A_start, l2, f, addremove=-1)

    # If lambda2 is unique, calculate other estimates as well.
    if lambda_mul == 1:
        results["2nd"][1] = second_approx_He(A_start, addremove=1)
        results["2nd"][-1] = second_approx_He(A_start, addremove=-1)

        results["exact"][1], errors["exact"][1] = exact_He(A_start, addremove=1)
        results["exact"][-1], errors["exact"][-1] = exact_He(A_start, addremove=-1)
    # Otherwise, calculate node centralities.
    else:
        G = nx.from_numpy_array(A_start)
        centrality = nx.closeness_centrality(G)

    # Numerically calculate real lambda2 changes.
    sensitivity_data = {}
    trend = {}
    for i in range(0, N):
        for j in range(i + 1, N):
            # Make a copy of the adjacency matrix to avoid modifying the original.
            sensitivity_data[(i, j)] = []
            A = A_start.copy()

            # Calculate continuous changes in l2 when changing the selected link (i, j).
            for t in x:
                A[i][j] = t
                A[j][i] = t
                l2, _, _ = get_fiedler(*get_laplacian(A))
                sensitivity_data[(i, j)].append(l2)
                trend[(i, j)] = "dash" if A_start[i][j] else "solid"

    # Make plots.
    fig = psub.make_subplots(rows=1, cols=2)
    nodes, edges = plotly_nx(nx.from_numpy_array(A_start))
    fig.add_trace(nodes, row=1, col=1)
    fig.add_trace(edges, row=1, col=1)

    def add_approximation_markers(data, pair_value, err_value, addremove, symbol, color):
        if addremove == 1:
            x = 1
            y = data[0] + pair_value
        elif addremove == -1:
            x = 0
            y = data[-1] + pair_value
        else:
            return

        err = None
        if err_value is not None:
            err = dict(type="data", array=[err_value])

        trace = pgo.Scatter(
            x=[x], y=[y], error_y=err, mode="markers", marker=dict(size=8, symbol=symbol, color=color), showlegend=False
        )
        return trace

    mem = set()  # Used to prevent plotting approximation markers multiple times.
    col_iter = itertools.cycle(px.colors.qualitative.Plotly)
    for edge in sensitivity_data:
        # Plot lines for the sensitivity data.
        color = next(col_iter)
        fig.add_trace(
            pgo.Scatter(x=x, y=sensitivity_data[edge], name=str(edge), mode="lines", line=dict(dash=trend[edge], color=color)),
            row=1,
            col=2,
        )

        # Plot markers for the approximations.
        for method in results:  # 1st approx., 2nd approx., exact formulation
            for action in results[method]:  # add or remove link
                id = (method, action)
                output = results[method][action]
                err = errors[method][action]
                if id not in mem and output is not None and output.best is not None and edge in output.best:
                    if err is None:
                        err = {}
                    trace = add_approximation_markers(
                        sensitivity_data[edge], output.pair[edge], err.get(edge, None), action, symbols[method], color
                    )
                    fig.add_trace(trace, row=1, col=2)
                    mem.add(id)

    # Add reference lines showing linear increase/decrease of l2 for a value of 1 (to see if changes are linear).
    fig.add_trace(
        pgo.Scatter(x=[0, 1], y=[l2_0, l2_0 + 1], name="+ref", mode="lines", line=dict(dash="dot", color="black")),
        row=1,
        col=2,
    )
    fig.add_trace(
        pgo.Scatter(x=[0, 1], y=[l2_0 - 1, l2_0], name="-ref", mode="lines", line=dict(dash="dot", color="black")),
        row=1,
        col=2,
    )

    # Add common legend entries for all methods.
    for method in results:
        fig.add_trace(
            pgo.Scatter(
                x=[0],
                y=[0],
                name=method,
                mode="markers",
                visible="legendonly",
                marker=dict(size=8, symbol=symbols[method], color="black"),
            ),
            row=1,
            col=2,
        )

    fig.update_layout(
        title=dict(text=f"Graph {k} (l2 = {l2_0}, f = {f})"),
        width=750,
        height=600,
        margin=dict(l=50, r=50, t=80, b=200),
    )

    # Print results.
    if lambda_mul == 1:
        def gen_annot(result):
            if result is None:
                return ""

            pairs = result.pair
            best = result.best
            return ", ".join(
                f"<b>{pair}={val}</b>" if pair in best else f"{pair}={val}" for pair, val in sorted(pairs.items())
            )

        annot_kwargs = dict(align="left", showarrow=False, xref="paper", yref="paper")
        fig.add_annotation(
            text="<b>Adding links</b><br>" + "<br>".join(gen_annot(results[key][1]) for key in sorted(results)),
            x=0.0,
            y=-0.3,
            **annot_kwargs,
        )
        fig.add_annotation(
            text="<b>Removing links</b><br>" + "<br>".join(gen_annot(results[key][-1]) for key in sorted(results)),
            x=0.0,
            y=-0.55,
            **annot_kwargs,
        )
    else:
        annot_kwargs = dict(align="left", showarrow=False, xref="paper", yref="paper")
        fig.add_annotation(text=f"Lambda2 multiplicity is {lambda_mul}.", x=0.0, y=-0.2, **annot_kwargs)
        fig.add_annotation(
            text="Centralities: <br>"
            + ", ".join(f"({k})={v:.2f}" for k, v in sorted(centrality.items(), key=lambda x: x[1])),
            x=0.0,
            y=-0.4,
            **annot_kwargs,
        )

    fig.show()


# A = np.array([
#     [0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
#     [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
#     [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#     [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
#     [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
#     [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
#     [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
#     [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1],
#     [0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
#     [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
#     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
#     [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
#     [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
#     [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1],
#     [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
# ], dtype=float
# )

# gs = {"A": A}

for k, A_start in enumerate(gs.values()):
    analyze_sensitiviy_and_approximations(k, A_start)

- This cell will plot the same graphs as the previous cell, but only for the first (Ghosh) approximation.  
- Additionally, it prints which links will be added/remove to maximize/minimize the change in lambda_2 (all four combinations). Then it shows the value of K_lambda_2 threshold for that edge (by default, it is 0.2 times the predicted change in lambda_2). 
- It also saves the figure in pdf.

In [ ]:
# HACK: https://github.com/microsoft/vscode-jupyter/issues/8131#issuecomment-1589961116
import plotly
from IPython.display import display, HTML

plotly.offline.init_notebook_mode()
display(
    HTML(
        '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
    )
)


def analyze_sensitiviy(k, A_start, p=False, save=False):
    x = np.linspace(0, 1, 50)
    N = A_start.shape[0]

    # Initial lambda value and max change estimate.
    lambdas, vectors, L = get_laplacian(A_start)
    init_lambda, _, f = get_fiedler(lambdas, vectors)

    addmax_est, addmax_links = calc_thresholds_combs(A_start, L, lambdas, vectors, np.nanmax, addremove=1, p=False)
    addmin_est, addmin_links = calc_thresholds_combs(A_start, L, lambdas, vectors, np.nanmin, addremove=1, p=False)
    remmax_est, remmax_links = calc_thresholds_combs(A_start, L, lambdas, vectors, np.nanmax, addremove=-1, p=False)
    remmin_est, remmin_links = calc_thresholds_combs(A_start, L, lambdas, vectors, np.nanmin, addremove=-1, p=False)

    sens_data = {}
    trend = {}
    for i in range(0, N):
        for j in range(i + 1, N):
            sens_data[(i, j)] = []
            A = A_start.copy()

            # Calculate continuous changes in l2 when changing links.
            for t in x:
                A[i][j] = t
                A[j][i] = t
                l2, _, _ = get_fiedler(*get_laplacian(A))
                sens_data[(i, j)].append(l2)
                trend[(i, j)] = "dash" if A_start[i][j] else "solid"

            # Something else

    # Make plots.
    fig = psub.make_subplots(rows=1, cols=2)
    # Left subplot - graph
    nodes, edges = plotly_nx(nx.from_numpy_array(A_start))
    fig.add_trace(nodes, row=1, col=1)
    fig.add_trace(edges, row=1, col=1)
    fig.update_xaxes(showticklabels=False, row=1, col=1)
    fig.update_yaxes(showticklabels=False, row=1, col=1)
    # Right subplot - sensitivity
    for edge in sens_data:
        fig.add_trace(
            pgo.Scatter(x=x, y=sens_data[edge], name=str(edge), mode="lines", line=dict(dash=trend[edge])), row=1, col=2
        )
    fig.update_xaxes(title_text="$\large\mathrm{Edge\ weight}\ a_{ij}$", title_standoff=5, row=1, col=2)
    fig.update_yaxes(title_text="$\large\mathrm{Algebraic\ connectivity}\ \lambda_2$", title_standoff=5, row=1, col=2)
    fig.update_layout(
        title=dict(text=f"Graph {k} (l2 = {init_lambda}, f = {f})"),
        width=900,
        height=650,
        margin=dict(l=50, r=50, t=80, b=250),
        font=dict(size=16),
    )

    # Print results.
    def gen_annot(est, links,):
        return (f'Estimated delta = {est}<br>'
                f'Links maximizing delta = {links}<br>')

    annot_kwargs = dict(align="left", showarrow=False, xref="paper", yref="paper")

    fig.add_annotation(
        text="<b>Adding max link</b><br>" + gen_annot(addmax_est, addmax_links), x=0.0, y=-0.4, **annot_kwargs
    )
    fig.add_annotation(
        text="<b>Adding min link</b><br>" + gen_annot(addmin_est, addmin_links), x=0.0, y=-0.65, **annot_kwargs
    )
    fig.add_annotation(
        text="<b>Removing max link</b><br>" + gen_annot(remmax_est, remmax_links), x=1, y=-0.4, **annot_kwargs
    )
    fig.add_annotation(
        text="<b>Removing min link</b><br>" + gen_annot(remmin_est, remmin_links), x=1, y=-0.65, **annot_kwargs
    )
    fig.show()

    if save:
        fig.write_image(f"l2_function_{k}.{save}")


for k, A_start in enumerate(gs.values()):
    if k in [3, 7]:
        analyze_sensitiviy(k, A_start, save="pdf")

# Check the consistency of $$\Delta \lambda_{2, max} = \max(f_i - f_j)^2$$

In [ ]:
from collections import Counter

count = Counter()
for k, A_start in enumerate(gs.values()):

    # Initial lambda value and max change estimate.
    lambdas, vectors, L = get_laplacian(A_start)
    init_lambda, _, f = get_fiedler(lambdas, vectors)
    delta_est, init_links = calc_thresholds_combs(A_start, L, lambdas, vectors, np.nanmax, addremove=1, p=False)

    # Go through all pairs of nodes and find the link that maximizes the change in lambda_2.
    n = A_start.shape[0]
    max_lambda = init_lambda
    max_links = set()
    for i in range(0, NV):
        for j in range(i + 1, NV):
            if A_start[i][j] == 0:
                A = A_start.copy()
                A[i][j] = 1
                A[j][i] = 1

                new_lambda, _, _ = get_fiedler(*get_laplacian(A))
                if (new_lambda - max_lambda) > 10e-5:
                    max_lambda = new_lambda
                    max_links = {(i, j)}
                elif abs(new_lambda - max_lambda) < 10e-5 and abs(new_lambda - init_lambda) > 10e-5:
                    max_links.add((i, j))

    print(f"Graph {k}...", end=" ")

    # Link(s) that maximize the change in lambda_2 are the same as the initial link(s) found by Ghosh.
    if max_links == init_links or not max_links or init_links.issubset(max_links):
        out = "OK"
    # Ghosh's estimate includes wrong links, but also the correct one.
    elif max_links.issubset(init_links):
        out = "Partial"
    # Approximation failed completely. Analyze why.
    else:
        out = "FAIL"
    count[out] += 1

    # Print the initial lambda_2, lambda_2 after adding the link, and the estimated and actual links.
    print(f"{out} (l2o={init_lambda}, l2f={max_lambda}, est={init_links}, act={max_links})", flush=True)

    if out == "FAIL":
        analyze_sensitiviy(k, A_start)

total = sum(count.values())
print("=========================")
print(f"OK     : {count['OK']}/{total} [{count['OK'] / total * 100:.2f} %]")
print(f"Partial: {count['Partial']}/{total} [{count['Partial'] / total * 100:.2f} %]")
print(f"FAIL   : {count['FAIL']}/{total} [{count['FAIL'] / total * 100:.2f} %]")